In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_7.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,There are many reasons why I think that the fa...,0.0
1,While I feel like this could help with a few t...,0.0
2,"Dear, Fellow Scientist at NASA\n\nI have heard...",0.0
3,"The article, ""Making Mona Lisa Smile"" by Nick ...",0.0
4,I had no idea that becoming a seagoing cowboy ...,0.0
...,...,...
17983,The using of this new technology called the Fa...,0.0
17984,The development of driverless cars is a good t...,0.0
17985,"Welcome my name is Luke, and I am here to tell...",0.0
17986,"Dear TEACHER_NAME,\n\nI think you should requi...",0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1810])
Attention Mask size: torch.Size([17988, 1810])
Vocabulary Size: 29908
Max Sequence Length: 1810


In [6]:
model=Sequential()
input_length=1810
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=128,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)

Epoch 1/5
450/450 [==============================] - 176s 388ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 177s 394ms/step - loss: 4.7211e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 177s 393ms/step - loss: 2.3827e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 169s 375ms/step - loss: 1.3961e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 170s 378ms/step - loss: 9.5134e-06 - accuracy: 1.0000


In [7]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 11s 92ms/step - loss: 7.9615e-06 - accuracy: 1.0000


[7.961541086842772e-06, 1.0]

In [9]:
model.save("newmodel7.keras")